In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import pandas as pd
import time

In [ ]:
def scroll_down(driver, scroll_times, scroll_distance=1000, scroll_pause_time=2):
    for _ in range(scroll_times):
        driver.execute_script(f"window.scrollBy(0, {scroll_distance});")
        time.sleep(scroll_pause_time)

In [ ]:
BASE_URL = ""

In [ ]:
def land_page(url):
    driver.get(url)

In [ ]:
driver = webdriver.Chrome()
driver.implicitly_wait(20)
land_page(BASE_URL)

In [ ]:
scroll_down(driver, 10,1000, 2)

In [ ]:
def get_comments():
    comments_section = driver.find_element(By.ID, 'contents')
    comments = comments_section.find_elements(By.TAG_NAME, 'ytd-comment-thread-renderer')
    data = []
    for i in comments:
        name = i.find_element(By.CSS_SELECTOR, 'h3.style-scope.ytd-comment-renderer').text
        comment = i.find_element(By.ID, 'comment-content').text
        data.append([name, comment])
    return data

In [ ]:
data = get_comments()

In [ ]:
df = pd.DataFrame(data, columns=['name', 'comments'])

In [ ]:
df.head(20)

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sia = SentimentIntensityAnalyzer()

# Function to determine sentiment label
def get_sentiment_label(sentiment_score):
    if sentiment_score > 0:
        return 1  # Positive
    elif sentiment_score < 0:
        return -1  # Negative
    else:
        return 0  # Neutral

In [ ]:
# Apply sentiment analysis and add sentiment column to DataFrame
df['sentiment_scores'] = df['comments'].apply(lambda x: sia.polarity_scores(x)['compound'])
df['sentiment'] = df['sentiment_scores'].apply(get_sentiment_label)

In [ ]:
df

In [ ]:
print(len(df))
print(df['sentiment'].value_counts())

In [ ]:
total_positive = (df['sentiment'] == 1).sum()
total_negative = (df['sentiment'] == -1).sum()
total_neutral = (df['sentiment'] == 0).sum()

# Calculate the total number of comments
total_comments = total_positive + total_negative + total_neutral

# Calculate the proportion of each sentiment category
proportion_positive = total_positive / total_comments
proportion_negative = total_negative / total_comments
proportion_neutral = total_neutral / total_comments

# Determine the overall sentiment based on the majority proportion
if proportion_positive > proportion_negative:
    overall_sentiment = "Positive"
elif proportion_positive < proportion_negative:
    overall_sentiment = "Negative"
else:
    overall_sentiment = "Neutral"

# Print or display the results
print(f"Total Comments: {total_comments}")
print(f"Proportion Positive: {proportion_positive:.2%}")
print(f"Proportion Negative: {proportion_negative:.2%}")
print(f"Proportion Neutral: {proportion_neutral:.2%}")
print(f"Overall Sentiment: {overall_sentiment}")

In [ ]:
df[df['sentiment'] == 0]['comments'][92]

In [ ]:
df.to_csv('comment.csv', index=False)